In [29]:
#我们只需要在全连接层后添加gluon.nn.Dropout层并指定元素丢弃概率。

from mxnet.gluon import nn
net = nn.Sequential()
drop_prob1=0.2
drop_prob2=0.5


with net.name_scope():
    net.add(nn.Flatten())
    #第一层全连接
    net.add(nn.Dense(256, activation="relu"))
    #添加丢弃层
    net.add(nn.Dropout(drop_prob1))
    #第二层全连接
    net.add(nn.Dense(256,activation="relu"))
    #添加丢弃层
    net.add(nn.Dropout(drop_prob2))
    net.add(nn.Dense(10))
net.initialize()


In [30]:
#read data
import sys
sys.path.append("..")
from mxnet import ndarray as nd
from mxnet import autograd
import utils

batch_size=256
train_data,test_data=utils.load_data_fashion_mnist(batch_size)




In [31]:
#define soft cost function 
from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.5})


In [32]:
#training
for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output=net(data)
            loss=softmax_cross_entropy(output,label)
            loss.backward()
            trainer.step(batch_size)

            
            train_loss += nd.mean(loss).asscalar()
            train_acc += utils.accuracy(output, label)

        test_acc = utils.evaluate_accuracy(test_data, net)
        print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))
        

MXNetError: [20:53:14] src/imperative/imperative.cc:192: Check failed: AGInfo::IsNone(*(outputs[i])) Assigning to NDArrays that are already in a computational graph will cause undefined behavior when evaluating gradients. Please call backward first to clear the graph or do this out side of a record section. 

Stack trace returned 10 entries:
[bt] (0) /home/wuying/miniconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x17c54c) [0x7f81c535c54c]
[bt] (1) /home/wuying/miniconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x23b1d8b) [0x7f81c7591d8b]
[bt] (2) /home/wuying/miniconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x22febb6) [0x7f81c74debb6]
[bt] (3) /home/wuying/miniconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(MXImperativeInvokeEx+0x63) [0x7f81c74dee93]
[bt] (4) /home/wuying/miniconda3/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7f81f4084ec0]
[bt] (5) /home/wuying/miniconda3/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call+0x22d) [